# 4. Improving and understanding algorithm performance.

Having built a basic machine learning pipeline, one wants to improve performance to get the best results, and also understand the results of the trained algorithm.

NOOA template sections
## Overview 

### Prerequisites
xx

### Learning Outcomes 
xx

## Tutorial 
Hyperparameter Tuning

Data Cleaning Augmentation
 - imbalance learn
 - resampling classes
 
Interpretability & explainability
https://github.com/djgagne/ams-ml-python-course module 4 deals with interpretability

Feature Importance

New metrics

Keeping track of results - ML Flow

## Excercise 
xx

## Examples of use
xx
 
## Next steps
xx

## Dataset Info
xx

## References
xx
